![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/PySpark/6.PySpark_Classification.ipynb)

# **PySpark Tutorial-6 Classification**

# **Overview** 

In this notebook, linear regression is performed for the titanic dataset using PySpark.


### **LOGISTIC REGRESSION**

*Logistic Regression is a Machine Learning algorithm which is used for the classification problems, it is a predictive analysis algorithm and based on the concept of probability.*


###  **Install spark**










In [ ]:
! pip install pyspark


### **Import Library**

> ###### Start spark session, read csv file and make some analysis about data



In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark2 = SparkSession.builder.appName('logistic').getOrCreate()

In [ ]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/PySpark/data/titanic.csv


In [ ]:
titanic = spark2.read.csv('titanic.csv', inferSchema=True, header=True)

In [ ]:
titanic.printSchema()

root
 |-- Passengerid: integer (nullable = true)
 |-- Age: double (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Sex: integer (nullable = true)
 |-- sibsp: integer (nullable = true)
 |-- zero5: integer (nullable = true)
 |-- zero6: integer (nullable = true)
 |-- zero7: integer (nullable = true)
 |-- zero8: integer (nullable = true)
 |-- zero9: integer (nullable = true)
 |-- zero10: integer (nullable = true)
 |-- zero11: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- zero13: integer (nullable = true)
 |-- zero14: integer (nullable = true)
 |-- zero15: integer (nullable = true)
 |-- zero16: integer (nullable = true)
 |-- zero17: integer (nullable = true)
 |-- zero18: integer (nullable = true)
 |-- zero19: integer (nullable = true)
 |-- zero20: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- zero22: integer (nullable = true)
 |-- zero23: integer (nullable = true)
 |-- Embarked: integer (nullable = true)
 |-- zero25: integer (nullable

In [ ]:
titanic.describe().show()

+-------+-----------------+------------------+-----------------+------------------+------------------+-----+-----+-----+-----+-----+------+------+------------------+------+------+------+------+------+------+------+------+------------------+------+------+------------------+------+------+------------------+
|summary|      Passengerid|               Age|             Fare|               Sex|             sibsp|zero5|zero6|zero7|zero8|zero9|zero10|zero11|             Parch|zero13|zero14|zero15|zero16|zero17|zero18|zero19|zero20|            Pclass|zero22|zero23|          Embarked|zero25|zero26|          2urvived|
+-------+-----------------+------------------+-----------------+------------------+------------------+-----+-----+-----+-----+-----+------+------+------------------+------+------+------+------+------+------+------+------+------------------+------+------+------------------+------+------+------------------+
|  count|             1309|              1309|             1309|              1

In [ ]:
titanic = titanic.withColumnRenamed('2urvived','Survived') # change the column name with withColumnRenamed command

In [ ]:
titanic.show(2)

+-----------+----+-------+---+-----+-----+-----+-----+-----+-----+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+--------+------+------+--------+
|Passengerid| Age|   Fare|Sex|sibsp|zero5|zero6|zero7|zero8|zero9|zero10|zero11|Parch|zero13|zero14|zero15|zero16|zero17|zero18|zero19|zero20|Pclass|zero22|zero23|Embarked|zero25|zero26|Survived|
+-----------+----+-------+---+-----+-----+-----+-----+-----+-----+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+--------+------+------+--------+
|          1|22.0|   7.25|  0|    1|    0|    0|    0|    0|    0|     0|     0|    0|     0|     0|     0|     0|     0|     0|     0|     0|     3|     0|     0|       2|     0|     0|       0|
|          2|38.0|71.2833|  1|    1|    0|    0|    0|    0|    0|     0|     0|    0|     0|     0|     0|     0|     0|     0|     0|     0|     1|     0|     0|       0|     0|     0|       1|
+-----------+----+--

In [ ]:
titanic.columns

['Passengerid',
 'Age',
 'Fare',
 'Sex',
 'sibsp',
 'zero5',
 'zero6',
 'zero7',
 'zero8',
 'zero9',
 'zero10',
 'zero11',
 'Parch',
 'zero13',
 'zero14',
 'zero15',
 'zero16',
 'zero17',
 'zero18',
 'zero19',
 'zero20',
 'Pclass',
 'zero22',
 'zero23',
 'Embarked',
 'zero25',
 'zero26',
 'Survived']

In many cases, NULL on columns needs to handles before you performing any operations on columns as operations on NULL values results in unexpected values.
https://sparkbyexamples.com/pyspark/pyspark-filter-rows-with-null-values/

In [ ]:
titanic.filter("Embarked is NULL").show()

+-----------+----+----+---+-----+-----+-----+-----+-----+-----+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+--------+------+------+--------+
|Passengerid| Age|Fare|Sex|sibsp|zero5|zero6|zero7|zero8|zero9|zero10|zero11|Parch|zero13|zero14|zero15|zero16|zero17|zero18|zero19|zero20|Pclass|zero22|zero23|Embarked|zero25|zero26|Survived|
+-----------+----+----+---+-----+-----+-----+-----+-----+-----+------+------+-----+------+------+------+------+------+------+------+------+------+------+------+--------+------+------+--------+
|         62|38.0|80.0|  1|    0|    0|    0|    0|    0|    0|     0|     0|    0|     0|     0|     0|     0|     0|     0|     0|     0|     1|     0|     0|    null|     0|     0|       1|
|        830|62.0|80.0|  1|    0|    0|    0|    0|    0|    0|     0|     0|    0|     0|     0|     0|     0|     0|     0|     0|     0|     1|     0|     0|    null|     0|     0|       1|
+-----------+----+----+---+-----+--

In [ ]:
titanic = titanic.filter("Embarked is not NULL")

##### **Import ML Libraries**

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
assembler2 = VectorAssembler(inputCols=['Age', 'Fare', 'Sex', 'Pclass', 'Embarked'], outputCol='features')

In [ ]:
output2 = assembler2.transform(titanic)

In [ ]:
final_data2 = output2.select('features', 'Survived')

In [ ]:
final_data2.show(5)

+--------------------+--------+
|            features|Survived|
+--------------------+--------+
|[22.0,7.25,0.0,3....|       0|
|[38.0,71.2833,1.0...|       1|
|[26.0,7.925,1.0,3...|       1|
|[35.0,53.1,1.0,1....|       1|
|[35.0,8.05,0.0,3....|       0|
+--------------------+--------+
only showing top 5 rows



##### **Train Test Split**

In [ ]:
train_titanic, test_titanic = final_data2.randomSplit([0.7,0.3])

In [ ]:
train_titanic.describe().show()

+-------+------------------+
|summary|          Survived|
+-------+------------------+
|  count|               923|
|   mean|0.2524377031419285|
| stddev|0.4346465080384674|
|    min|                 0|
|    max|                 1|
+-------+------------------+



In [ ]:
test_titanic.describe().show()

+-------+------------------+
|summary|          Survived|
+-------+------------------+
|  count|               384|
|   mean|0.2786458333333333|
| stddev|0.4489177465504924|
|    min|                 0|
|    max|                 1|
+-------+------------------+



In [ ]:
lr_titanic = LogisticRegression(labelCol='Survived')

In [ ]:
fitted_titanic_model = lr_titanic.fit(train_titanic)

In [ ]:
training_sum = fitted_titanic_model.summary

In [ ]:
training_sum.predictions.describe().show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+-------+------------------+-------------------+
|summary|          Survived|         prediction|
+-------+------------------+-------------------+
|  count|               923|                923|
|   mean|0.2524377031419285|0.18418201516793067|
| stddev|0.4346465080384674|  0.387842714477069|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
pred_and_labels = fitted_titanic_model.evaluate(test_titanic)

In [ ]:
pred_and_labels.predictions.show()

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


+--------------------+--------+--------------------+--------------------+----------+
|            features|Survived|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|[0.67,14.5,0.0,2....|       1|[0.98091631419756...|[0.72728999526542...|       0.0|
|[0.75,13.775,0.0,...|       0|[1.90764968707244...|[0.87075487210722...|       0.0|
|[0.83,9.35,0.0,3....|       0|[1.90074718196415...|[0.86997606851584...|       0.0|
|[1.0,12.1833,1.0,...|       0|[-0.1613873069630...|[0.45974051809542...|       1.0|
|[1.0,16.7,1.0,3.0...|       0|[-0.1517628760810...|[0.46213193444270...|       1.0|
|[1.0,37.0042,0.0,...|       1|[0.72526087078323...|[0.67376444299566...|       0.0|
|[2.0,21.075,0.0,3...|       0|[1.96268190064421...|[0.87682290262338...|       0.0|
|[2.0,31.275,1.0,3...|       0|[-0.0891240904662...|[0.47773371405046...|       1.0|
|[2.0,39.6875,0.0,...|       0|[2.00234243151677...|[0.8810427989

In [ ]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='Survived')

In [ ]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
auc

# The Area Under the Curve (AUC) is the measure of the ability of a classifier to distinguish between classes and is used as a summary of the ROC curve.
# The higher the AUC, the better the performance of the model at distinguishing between the positive and negative classes.

# https://www.analyticsvidhya.com/blog/2020/06/auc-roc-curve-machine-learning/#:~:text=The%20Area%20Under%20the%20Curve,the%20positive%20and%20negative%20classes.


0.6403050035426296